In [1]:
from typing import List, Optional

from pydantic import BaseModel
from langchain_google_vertexai import ChatVertexAI

from utils import BaseXMLModel
from IPython.display import Markdown

In [2]:
class Section(BaseXMLModel):
    title: str
    description: Optional[str] = None
    required: bool = True

class Format(BaseXMLModel):
    topic: str
    audience: str
    description: str
    sections: List[Section]

In [3]:
newsletter = Format(
    topic='AI and Large Language Models',
    audience='technical, mostly sofware engineers and data scientists',
    description='Newsletter of latest developments in the field',
#    required_sections=['TL;DR - summary of main development'],
    sections=[
        Section(title='TL;DR', description='summary of main development'),
        Section(title='Name of the article', description='one section for each article within selected topic', required=False)
    ]
)

In [7]:
newsletter.to_xml()

"<Format>\n<topic>AI and Large Language Models</topic><audience>technical, mostly sofware engineers and data scientists</audience><description>Newsletter of latest developments in the field</description><sections>\n<ITEM>{'title': 'TL;DR', 'description': 'summary of main development', 'required': True}</ITEM>\n<ITEM>{'title': 'Name of the article', 'description': 'one section for each article within selected topic', 'required': False}</ITEM>\n</sections>\n</Format>"

In [5]:
model = ChatVertexAI(model='gemini-1.5-pro-002')

In [8]:
prompt = f"""Create outline of document with following specifications: {newsletter.to_xml()}
<ARTICLES>
<ARTICLE ID='e31'>
<TITLE>Greate use-cases for LLMs in Financial Services</TITLE>
<SUMMARY>Inspiration for financial services profesionals how to use LLMs in their work</SUMMARY>
</ARTICLE>
<ARTICLE ID='d12'>
<TITLE>Evaluation of LLMs</TITLE>
<SUMMARY>Summary of evaluation techiques for GenAI applications</SUMMARY>
</ARTICLE>
<ARTICLE ID='bab'>
<TITLE>Mars colony plans</TITLE>
</ARTICLE>
</ARTICLES>

Respond with list of sections, each section with plan how to create content for it.
"""

In [9]:
response = model.invoke([("user", prompt)])

In [10]:
Markdown(response.content)

```json
[
  {
    "title": "TL;DR",
    "content_plan": "Summarize the key takeaways from the two selected articles ('Great use-cases for LLMs in Financial Services' and 'Evaluation of LLMs').  Focus on practical applications and advancements in evaluation techniques.  Example:  This week's newsletter covers practical applications of LLMs in financial services, including [mention a specific use case or two]. We also explore the latest techniques for evaluating GenAI models, highlighting [mention a key technique or challenge].",
    "required": true
  },
  {
    "title": "Great use-cases for LLMs in Financial Services",
    "content_plan": "Extract key use-cases presented in article 'e31'.  Provide brief descriptions for each use-case, potentially categorizing them (e.g., by business function, risk level, or implementation complexity).  Consider including code examples or links to relevant resources if available in the original article. Target audience is technical, so go beyond general statements and provide more technical details or architectural considerations where possible.",
    "article_id": "e31",
    "required": false
  },
  {
    "title": "Evaluation of LLMs",
    "content_plan": "Summarize the evaluation techniques presented in article 'd12'. Focus on explaining *how* these techniques work and *why* they are relevant. Provide examples of metrics used. Consider including short code snippets (e.g., Python) to demonstrate how these techniques might be implemented. Since the target audience is technical, including more details and practical examples is important.",
    "article_id": "d12",
    "required": false
  }
]
```
